# New thermocycling ML

## copying old stuff and examples

### Data import

### SavGol

In [ ]:
def SavGol(trace: np.ndarray, window_length: int = 600, polyorder: int = 3) -> np.ndarray:
    baseline_trend = savgol_filter(x = trace, window_length = window_length, polyorder = polyorder, mode = 'mirror')
    return trace - baseline_trend

def LOWESS(trace: pd.Series, cycles: int = 200) -> np.ndarray:
    if cycles >= len(trace): print("data do not have enough cycles for this 'cycles' parameter")
    else:
        baseline_trend = lowess(trace.values, trace.index, frac=cycles/len(trace), return_sorted=False)
        return trace.values - baseline_trend
def SavGoldf(df_: pd.DataFrame,
             voltage_point: int = 201,
             window_length: int = 600,
             polyorder: int = 4) -> pd.DataFrame:
    '''voltage point: number of the column, corresponding to certain voltage
    window_length: smoothing parameter. Typically 200-800
    polyorder: 3-4'''
    data = df_.iloc[:, :402].values
    envelope = df_.iloc[:, voltage_point].values
    baseline = savgol_filter(x=envelope,
                             window_length=window_length,
                             polyorder=polyorder,
                             mode='mirror')
    baseline_df = [[x] * 402 for x in envelope]
    corrected_df = pd.DataFrame(data=data - baseline_df,
                                index=df_.index.values,
                                columns=list(df_.columns)[:402])
    corrected_df = pd.concat([corrected_df, df_.iloc[:,-5:]], axis=1)
    return corrected_df
def LOWESSdf(df_: pd.DataFrame,
             voltage_point: int = 201,
             window_length: int = 600,
             polyorder: int = 4) -> pd.DataFrame:
    '''voltage point: number of the column, corresponding to certain voltage
    window_length: smoothing parameter. Typically 200-800
    polyorder: 4'''
    data = df_.iloc[:, :402].values
    envelope = df_.iloc[:, voltage_point].values
    index = df_.index.values
    baseline = lowess(endog=envelope,
                      exog=index,
                      frac=window_length / df_.shape[0],
                      return_sorted=False)
    baseline_df = [[x] * 402 for x in envelope]
    corrected_df = pd.DataFrame(data=data - baseline_df,
                                index=df_.index.values,
                                columns=list(df_.columns)[:402])
    corrected_df = pd.concat([corrected_df, df_.iloc[:,-5:]], axis=1)
    return corrected_df

### Optuna

In [ ]:
class CatBoostPruningCallback(object):

    def __init__(
        self, trial: optuna.trial.Trial, metric: str, valid_name: str = "validation"
    ) -> None:
        self._trial = trial
        self._metric = metric
        self._valid_name = valid_name
        self._pruned = False
        self._message = ""

    def after_iteration(self, info) -> bool:
        step = info.iteration - 1
        if self._valid_name not in info.metrics:
            raise ValueError(
                'The entry associated with the validation name "{}" '
                "is not found in the evaluation result list {}.".format(self._valid_name, info)
            )
        metrics = info.metrics[self._valid_name]
        if self._metric not in metrics:
            raise ValueError(
                'The entry associated with the metric name "{}" '
                "is not found in the evaluation result list {}.".format(self._metric, info)
            )
        current_score = metrics[self._metric][-1]
        self._trial.report(current_score, step=step)
        if self._trial.should_prune():
            self._message = "Trial was pruned at iteration {}.".format(step)
            self._pruned = True
            return False
        return True

    def check_pruned(self) -> None:
        """Check whether pruend."""
        if self._pruned:
            raise optuna.TrialPruned(self._message)

In [ ]:
def objective(trial: optuna.Trial) -> float:
    train_x, valid_x, train_y, valid_y = train_test_split(dfc_pca, pd.Series(labels), test_size=.33, stratify=labels)

    param = {
        "objective": "MAE",  # trial.suggest_categorical("objective", ["MAE"])
        "iterations": trial.suggest_int("iterations", 100, 3000, step = 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.4, log = True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.5, 5, log = True),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.1, 1, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "boosting_type": "Plain", # trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),  # 
        "bootstrap_type": "MVS",  # trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),  # 
        #"subsample": trial.suggest_float("subsample", 0.1, 1, log=True),  # only valid for Bernoulli
        "used_ram_limit": "8gb",
        "eval_metric": "MAE",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = CatBoostRegressor(**param)

    pruning_callback = CatBoostPruningCallback(trial, "MAE", "validation")
    gbm.fit(train_x, train_y, 
            eval_set=[(valid_x, valid_y)],
            verbose=0,
            early_stopping_rounds=500,
            callbacks=[pruning_callback],
           )
    pruning_callback.check_pruned()
    
    return gbm.get_best_score()['validation']['MAE']


if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"  # pruners.SuccessiveHalvingPruner()
    )
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value:.4f}")

    print("  Params: ")
    for key, value in trial.params.items():
        if type(value) == float:
            print(f"    {key}: {value:.4f}")
        else:
            print(f"    {key}: {value}")

In [ ]:
# Multioutput regression
def objective(trial: optuna.Trial) -> float:
    train_x, valid_x, train_y, valid_y = train_test_split(dfc_pca, multiregression_labels, test_size=.33, stratify=multiregression_labels)

    param = {
        "objective": "MultiRMSE",  # trial.suggest_categorical("objective", ["MAE"])
        "iterations": trial.suggest_int("iterations", 100, 2000, step = 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.5, log = True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.5, 7, log = True),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.1, 1, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "boosting_type": "Plain", # trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),  # 
        "bootstrap_type": "MVS",  # trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),  # 
        #"subsample": trial.suggest_float("subsample", 0.1, 1, log=True),  # only valid for Bernoulli
        "used_ram_limit": "8gb",
        "eval_metric": "MultiRMSE",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = CatBoostRegressor(**param)

    pruning_callback = CatBoostPruningCallback(trial, "MultiRMSE", "validation")
    gbm.fit(train_x, train_y, 
            eval_set=[(valid_x, valid_y)],
            verbose=0,
            early_stopping_rounds=100,
            callbacks=[pruning_callback],
           )
    pruning_callback.check_pruned()
    
    return gbm.get_best_score()['validation']['MultiRMSE']


if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"  # pruners.SuccessiveHalvingPruner()
    )
    study.optimize(objective, n_trials=1000, show_progress_bar = True)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value:.4f}")

    print("  Params: ")
    for key, value in trial.params.items():
        if type(value) == float:
            print(f"    {key}: {value:.4f}")
        else:
            print(f"    {key}: {value}")

In [ ]:
optuna.visualization.plot_param_importances(study).update_layout(height = 300).show()
optuna.visualization.plot_param_importances(study, target=lambda t: t.duration.total_seconds(), target_name="duration").update_layout(height = 300).show()
optuna.visualization.plot_slice(study).show()

### NN

In [ ]:
# Импортируем сам keras
import keras
# Последовательный тип модели
from keras.models import Sequential
# Импортируем полносвязный слой, слои активации и слой, превращающий картинку в вектор
from keras.layers import Dense, Activation, Flatten, LSTM, Embedding
from keras.utils import plot_model
import tensorflow as tf

In [ ]:
# time splitting data
dataset_time_split = 8
X_train_ = np.array(df.loc[df['meas_cycle'] < dataset_time_split].iloc[:,:402])
y_train = np.array(df.loc[df['meas_cycle'] < dataset_time_split].loc[:,gas])
X_test_ = np.array(df.loc[df['meas_cycle'] >= dataset_time_split].iloc[:,:402])
y_test = np.array(df.loc[df['meas_cycle'] >= dataset_time_split].loc[:,gas])
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train_)
X_test = scaler.transform(X_test_)
print(f"X train {X_train.shape}\nX test {X_test.shape}\ny train {y_train.shape}\ny test {y_test.shape}")

In [ ]:
#Scaling and encoding
encoder = LabelEncoder()
labels = keras.utils.to_categorical(encoder.fit_transform(labels), 4)
X_train_, X_test, y_train_, y_test = train_test_split(dfc, labels, test_size=.15, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_, test_size=.25, stratify=y_train_)

scaler = MinMaxScaler()
X_val = scaler.fit_transform(np.array(X_val))
X_train = scaler.fit_transform(np.array(X_train))
X_test = scaler.transform(np.array(X_test))

print(f"X train {X_train.shape}\nX val {X_val.shape}\nX test {X_test.shape}\ny train {y_train.shape}\ny val {y_val.shape}\ny test {y_test.shape}")
#Simple model

model = Sequential()

# Добавляем скрытый полносвязный слой из 64 нейронов
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[-1],)))

#model.add(Dense(units=32, activation='relu'))
#model.add(Dense(units=16, activation='relu'))
# И активацию для скрытого слоя нейронов

# Добавляем выходной полносвязный слой из 4 нейронов
model.add(Dense(units=4, activation='softmax'))
#model.add(Dense(units=1, activation='softmax'))

# Чтобы получить на выходе вероятности для каждого класса, выбираем активацию
# softmax
model.summary()
# Компилируем модель
model.compile(loss='categorical_crossentropy',  # 'categorical_crossentropy' 'mean_squared_error'
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics='accuracy')  # [tf.keras.metrics.RootMeanSquaredError()]
history = model.fit(X_train, y_train, 
                    epochs=1000, batch_size=34,
                    validation_data=(X_test, y_test)
                    )
history = model.fit(X_train, y_train, 
                    epochs=1000, batch_size=34,
                    validation_data=(X_test, y_test),
                    )
# early stopping

from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", min_delta=.001,
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)
  
history = model.fit(X_train, y_train, batch_size = 201, 
                    epochs = 1000, validation_data =(X_val, y_val), 
                    callbacks =[earlystopping])
hdf = pd.DataFrame(history.history)
fig = px.line(hdf, x=hdf.index, y=['loss', 'val_loss', 'accuracy', 'val_accuracy'])  # 'root_mean_squared_error''loss', 'val_loss' 'accuracy', 'val_accuracy'
fig.show()
model.evaluate(X_test, y_test)
# Метод predict модели возвращает выходные значения последнего слоя нейросети 
y_test_predictions = model.predict(X_test)
y_predicted_classes=np.argmax(y_test_predictions, axis=1)
y_real_classes=np.argmax(y_test, axis=1)
y_true = encoder.inverse_transform(y_real_classes)
y_pred = encoder.inverse_transform(y_predicted_classes)
fig, ax = plt.subplots(figsize=(3,3), dpi=150)
class_labels = ['Acetone', 'H2S', 'NO2', 'air']
fig = ConfusionMatrixDisplay.from_predictions(y_true, y_pred, normalize = 'pred', ax = ax, xticks_rotation=30, cmap = 'viridis')
ax.set_title(f'NO2 (128-32) NN\n accuracy = {model.evaluate(X_test, y_test)[1]:.2f}of')
plt.show()

### RNN

In [ ]:
# Write a timesplit here befor aplying LSTM
def create_lstm_dataset(data, labels, look_back=1):
    dataX, dataY = [], []
    for i in range(len(data)-look_back-1):
        a = data[i:(i+look_back), :]  # take a batch of data - several rows
        dataX.append(a)
        dataY.append(labels[i + look_back - 1])  # predict the gas values of the last row taken
    return np.array(dataX), np.array(dataY)  # returns an array of shape [samples, time steps, features]

In [ ]:
trainX, trainY = create_lstm_dataset(X_train, y_train, look_back=look_back)
testX, testY = create_lstm_dataset(X_test, y_test, look_back=look_back)

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(16, input_shape=(look_back, 402)))
model.add(Dense(1))
model.compile(loss=loss, optimizer='adam')
model.summary()

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (look_back, 402)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss=loss, optimizer='adam')
model.summary()

In [ ]:
%%time
history = model.fit(trainX, trainY, epochs=200, batch_size=50, verbose=1, 
                    validation_data = (testX, testY))

In [ ]:
hdf = pd.DataFrame(history.history)
fig = px.line(hdf, x=hdf.index, y=['loss', 'val_loss'])  # 'loss', 'val_loss' 'accuracy', 'val_accuracy'
fig.show()

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
#trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
#testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0,:]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0,:]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
model = Sequential()
# Добавим слой Embedding ожидая на входе словарь размера 1000, и
# на выходе вложение размерностью 64.
#model.add(Embedding(input_dim=X_train.shape[-1], output_dim=32))

look_back = 20
# Добавим слой LSTM с 128 внутренними узлами.
model.add(LSTM(units=64, activation='tanh', input_shape=(1,look_back)))
#model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
model.summary()

## Library import

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.express.colors import sample_colorscale
pd.options.plotting.backend = 'plotly'
from sklearn.preprocessing import minmax_scale, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from scipy.signal import savgol_filter
import optuna
#import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

### Extra libraries

In [ ]:
import pandas as pd
import numpy as np
import os
from typing import Any
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.express.colors import sample_colorscale
pd.options.plotting.backend = "plotly"

In [ ]:
from sklearn.preprocessing import minmax_scale, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, Normalizer, PowerTransformer, MaxAbsScaler, LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.manifold import TSNE, Isomap, MDS, LocallyLinearEmbedding
from sklearn.metrics import confusion_matrix, classification_report, f1_score, mean_squared_error, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit, LeaveOneGroupOut, TimeSeriesSplit
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.signal import savgol_filter, convolve
from statsmodels.tsa.filters.filtertools import convolution_filter

In [ ]:
from catboost import CatBoostClassifier, Pool, CatBoost, CatBoostRegressor
import shap
import optuna
from optuna.samplers import NSGAIISampler
from optuna.integration import CatBoostPruningCallback
shap.initjs()

## Loading data

In [ ]:
def folder_reader(path: str, format_: str = 'brotli') -> (list, list):
    '''automatically detects if the path is long or short (starts with OneDrive/...)
       if short, correctly extends the path to desired folder with C:/%username%/...
       checks the orientation of slashes, changes it to "/" and adds "/" in the end
       then returns two lists of files of the chosen format (.brotli, .xls) sorted by date
       data_list = filenames
       data_path_list = full paths
    '''
    path.replace('\\', '/')
    if not path.endswith("/"):
        path += "/"
    full_path = path if path.startswith("C:/") else 'C:/Users/' + os.environ.get("USERNAME") + '/' + path
    data_list = [str(file)[len(full_path):] for file in
                 sorted(Path(full_path).iterdir(), key=os.path.getmtime, reverse=True) if str(file).endswith(format_)]
    data_path_list = [full_path + x for x in data_list]
    data_list_df = pd.DataFrame(data=data_list,
                                columns=['file names'],
                                index=np.arange(len(data_list)))
    print(data_list_df)
    return data_list, data_path_list

In [ ]:
path_to_data = r'YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021'
data_list, data_path_list = folder_reader(path_to_data, '.brotli')

In [ ]:
def load_data(gas: str = 'NO2'):
    gas_path_dict = {'NO2': r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021\NO2_15_02_protocol10-15-25.brotli",
                     'H2S': r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021\H2S_17_02_protocol10-15-25.brotli",
                     'Acet': r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021\Acet_19_02_protocol10-15-25.brotli",
                     'NO2 ': r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021\NO2_16_04_protocol10-15-25.brotli",}
    df = pd.read_parquet(gas_path_dict[gas])
    if gas == 'H2S':
        df = df[15*402:]
    elif gas == 'NO2':
        df = df[:-400]
    elif gas == 'NO2 ':
        df = df[:5*120*402]
    else:
        pass
    switch_mask = (df['MFC_target'] == 0) & (df['MFC_target'].shift() == 25)
    incremental_values = switch_mask.cumsum()
    return df.assign(meas_cycle = incremental_values)
def true_gas(df: pd.DataFrame):
    #errors = df.flow_target_error.abs().values
    return df.MFC_target.values - df.flow_target_error.values
def SavGol(trace, window_length: int = 600, polyorder: int = 3) -> np.ndarray:
    baseline_trend = savgol_filter(x = trace, window_length = window_length, polyorder = polyorder, mode = 'mirror')
    return trace - baseline_trend
def cut_into_Vpulses(data_column: np.array, gas_column: np.array, meas_cycle_column: np.array, gas: str, dp_per_pulse: int = 402): 
    
    gas_column = gas_column.reshape((len(data_column)//dp_per_pulse, dp_per_pulse)).mean(axis=1)
    meas_cycle = meas_cycle_column[::dp_per_pulse]
    if len(data_column) % dp_per_pulse != 0:
        print(f"df shape is not divisible by {dp_per_pulse}\n{len(data_column) % dp_per_pulse}")
    else:
        datadf = data_column.reshape((len(data_column)//dp_per_pulse, dp_per_pulse))
        zeros = np.zeros(len(data_column)//dp_per_pulse)
        datadf = pd.DataFrame(datadf).assign(NO2 = zeros,
                                             H2S = zeros,
                                             Acet = zeros, 
                                             meas_cycle = meas_cycle)
        datadf.loc[:,gas] = gas_column
        return datadf.assign(gas=gas)

In [ ]:
def forward_backward_exponential_smoothing(data: np.ndarray, alpha: float) -> np.ndarray:
    """
    Apply forward-backward exponential smoothing to data.

    Parameters:
        data (array_like): The input data to be smoothed.
        alpha (float): Smoothing factor between 0 and 1.

    Returns:
        smoothed_data (ndarray): Smoothed data.
    """
    if alpha >=1 or alpha <= 0:
        raise ValueError("Enter alpha such that 0 < alpha < 1")
    n = len(data)
    smoothed_data = np.zeros_like(data, dtype=float)

    # Forward pass
    smoothed_data[0] = data[0]
    for i in range(1, n):
        smoothed_data[i] = alpha * data[i] + (1 - alpha) * smoothed_data[i - 1]

    # Backward pass
    for i in range(n - 2, -1, -1):
        smoothed_data[i] = alpha * smoothed_data[i] + (1 - alpha) * smoothed_data[i + 1]

    return smoothed_data

In [ ]:
def dedrift_n_cut(df, method: str, how: str, window_length: int, alpha: float = 0.015, gas: str = 'NO2'):
    '''how - choose one of multienv, topenv, full to dedrift accordingly. Any other value will not dedrift data at all'''
    data = df.I.values
    gas_values = true_gas(df)
    meas_cycle_column = df.meas_cycle.values
    # dedrifting method
    if method == 'SavGol':
        filter_method = lambda x: savgol_filter(x=x, window_length = window_length, polyorder = 3, mode = 'mirror')
    elif method == 'exp':
        filter_method =  lambda x: forward_backward_exponential_smoothing(x, alpha)
    elif method == 'none':
        filter_method = lambda x: np.array([np.nanmean(x)]*len(x)).flatten()
    else: 
        raise ValueError(f"Currently only `exp` and `SavGol` methods of dedrifting are available. You entered {method}")
    
    #envelope to correct
    if how == 'topenv':
        envelopes = [201]
    elif how == 'multienv':
        envelopes = [50, 150, 201, 300]
    elif how == 'none':
        envelopes = [201]
    #elif how == 'full':  # to be avoided since it takes a lot of time
    #    dedrifted = SavGol(data, window_length=window_length * 402)
    else: 
        raise ValueError(f"Currently only `topenv` and `multienv` (and `none`) envelope choices are available. You entered {how}")
    env_dict = {}
    for envelope in envelopes:
        env_dict[envelope] = filter_method(data[envelope::402])
    envelope_ = np.array([[x]*402 for x in pd.DataFrame(env_dict).values.mean(axis=1)]).flatten()
    dedrifted = data - envelope_
    # cutting
    return cut_into_Vpulses(dedrifted, gas_values, meas_cycle_column, gas, dp_per_pulse=402)

def load_full_dedrifted_dataset(dedrifting_method: str = 'exp', envelope_choice: str = 'multienv', window_length: int = 350, alpha: float = 0.022):
    '''dedrifting method - choose one of `SavGol`, `exp`, `None`
    envelope_choice - choose one of `topenv`, `multienv`
    window_length - be careful not to exceed the sample size'''
    DF = pd.DataFrame()
    for gas in ['NO2', 'H2S', 'Acet']:
        df = dedrift_n_cut(load_data(gas), method=dedrifting_method, how=envelope_choice, window_length=window_length, alpha=alpha, gas=gas)
        DF = pd.concat([DF, df], ignore_index=True)
    return DF

In [ ]:
fig = go.Figure()
#alpha = 0.2
data = load_data("H2S").I.values
index = np.arange(len(data))
fig.add_trace(go.Scattergl(x = np.arange(len(data)//6).tolist(), y = data[::6].tolist(), name="raw", mode='lines'))
for alpha in [0.01, 0.015, 0.02, 0.025, 0.03]:
    envelope = np.array([[x]*402 for x in forward_backward_exponential_smoothing(data[201::402], alpha)]).flatten()
    fig.add_trace(go.Scattergl(x = np.arange(len(data)//6).tolist(), y = envelope[::6].tolist(), name=f"alpha = {alpha}", mode='lines'))
fig.show()
print()

In [ ]:
fig = go.Figure()
alpha = 0.2
data = load_data("NO2").I.values
index = np.arange(len(data))
fig.add_trace(go.Scattergl(x=np.arange(len(data)//6).tolist(), y=data[::6].tolist(), name="raw", mode='lines'))
envelope = np.array([[x]*402 for x in forward_backward_exponential_smoothing(data[201::402], alpha)]).flatten()
fig.add_trace(go.Scattergl(x = np.arange(len(data)/6), y = envelope[::6], name=f"alpha = {alpha}", mode='lines'))
fig.show()

In [ ]:
DF = load_full_dedrifted_dataset('SavGol', 'multienv', window_length=500, alpha=0.015)
print(DF.shape)

In [ ]:
DF

In [ ]:
DF.Acet.apply(lambda x: round(x,0)).unique()

## Splitting for train and test

In [ ]:
def sample_weight(gas, look_back: int = 500, to_weight: bool = True):
    '''function weights samples according to their TLV (the lower the TLV the higher the weight)
    and according to the total count of samples in the respective measurement 
    currently cannot figure out how to weight samples by both. Left only TLV weighting
    Almost figured that out, but somehow I constantly get 1-2% extra which is not good'''
    gas_counts = DF.gas.value_counts().to_dict()
    gas_tlvs = dict(NO2 = 1/0.2, H2S = 1/1, Acet = 1/250) if to_weight else dict(NO2=1, H2S=1, Acet=1) # TLV-TWA in ppm according to ACGIH
    sum_counts = sum(gas_counts.values()) - look_back*len(gas_tlvs.keys())
    sum_rec_TLV = sum(gas_tlvs.values())
    return gas_tlvs[gas] / sum_rec_TLV  # * (gas_counts[gas] - look_back) / sum_counts * len(gas_tlvs.keys())

In [ ]:
def scale_n_PCA(train: np.array, test: np.array, n_components: int, scale: bool= True, scaler = MinMaxScaler(), do_PCA: bool = True):
    if scale:
        scaler = MinMaxScaler() if scaler==None else scaler
        train = scaler.fit_transform(train)
        test = scaler.transform(test)
    if do_PCA:
        pca = PCA(n_components=n_components)
        train_pca = pca.fit_transform(train)
        test_pca = pca.transform(test)
        return train_pca, test_pca
    else:
        return train, test

In [ ]:
def train_test_TS(df_: pd.DataFrame, n_components: int = 5, start: int = 1) -> (list, list, list, list):
    '''start: int, meas_cycle to start with
    returns 4 lists for train/test x/y. Each element = split'''
    if not 'meas_cycle' in df_.columns:
        print("wrong dataframe: no meas_cycle column")
        return None
    else:
        n_splits = len(df_.meas_cycle.unique())
        train_X, test_X, train_y, test_y = [], [], [], []
        for split in range(start,n_splits-1):  # starting from 1 cause the first meas cycle was cut
            # PCA
            train = df_.loc[df_['meas_cycle']<=split].iloc[:,:402].values
            test = df_.loc[df_['meas_cycle']==split+1].iloc[:,:402].values
            train_pca, test_pca = scale_n_PCA(train, test, n_components=n_components, scale=True, scaler=MinMaxScaler(), do_PCA=True)
            train_X.append(train_pca)
            train_y.append(df_.loc[df_['meas_cycle']<=split].iloc[:,402:405].values)
            test_X.append(test_pca)
            test_y.append(df_.loc[df_['meas_cycle']==split+1].iloc[:,402:405].values)
        return train_X, test_X, train_y, test_y

In [ ]:
def train_test_TS_class(df_: pd.DataFrame, n_components: int = 5, start: int = 1) -> (list, list, list, list):
    '''start: int, meas_cycle to start with
    returns 4 lists for train/test x/y. Each element = split'''
    if not ('meas_cycle' or 'class_') in df_.columns:
        raise ValueError("wrong dataframe: no meas_cycle or class_ column")
    else:
        n_splits = len(df_.meas_cycle.unique())
        train_X, test_X, train_y, test_y = [], [], [], []
        for split in range(start,n_splits-1):  # starting from 1 cause the first meas cycle was cut
            # PCA
            train = df_.loc[df_['meas_cycle']<=split].iloc[:,:402].values
            test = df_.loc[df_['meas_cycle']==split+1].iloc[:,:402].values
            train_pca, test_pca = scale_n_PCA(train, test, n_components=n_components, scale=True, scaler=MinMaxScaler(), do_PCA=True)
            train_X.append(train_pca)
            train_y.append(df_.loc[df_['meas_cycle']<=split].loc[:,"class_"].values)
            test_X.append(test_pca)
            test_y.append(df_.loc[df_['meas_cycle']==split+1].loc[:,"class_"].values)
        return train_X, test_X, train_y, test_y

In [ ]:
def add_class_column(df: pd.DataFrame):
    if not all(col in df.columns for col in ['NO2', 'H2S', 'Acet', 'gas']):
        raise ValueError("DataFrame is missing required columns: 'NO2', 'H2S', 'Acet', 'gas'")
    
    df['class_'] = df.apply(lambda row: 'air' if row['NO2'] + row['H2S'] + row['Acet'] == 0 else row['gas'], axis=1)
    return df

In [ ]:
def create_RNN_sequences(data: np.ndarray, look_back: int = 50):
    num_samples, num_features = data.shape
    if num_samples//4*3 < look_back:
        look_back = int(num_samples//4*3)
        print(f"Look back too large, number of samples is {num_samples}. Lookback is set to {look_back}")
    num_sequences = num_samples - look_back + 1
    #print(num_sequences, look_back, num_features)
    sequences = np.zeros((num_sequences, look_back, num_features))
    for i in range(look_back):
        sequences[:, i] = data[i:num_samples - look_back + 1 + i]
    return list(sequences), look_back

In [ ]:
def create_RNN_sequences_for_multiple_gases(X: np.ndarray, y: np.ndarray, gas_type: np.ndarray, look_back: int = 50, to_weight: bool = True) -> (np.ndarray, np.ndarray):
    df_ = pd.concat([pd.DataFrame(X), pd.DataFrame(y)], axis = 1, ignore_index=True).assign(gas_ = gas_type)
    big_sequence_X, big_sequence_y = [], []
    sample_weights = []
    for gas in df_.gas_.unique():
        sequence_X, new_look_back = create_RNN_sequences(data = df_.loc[df_['gas_'] == gas].iloc[:,:X.shape[1]].values, look_back=look_back)
        sequence_y = df_.loc[df_['gas_']==gas].iloc[new_look_back - 1:, X.shape[1]:X.shape[1] + y.shape[1]].values
        big_sequence_X.extend(list(sequence_X))
        big_sequence_y.extend(list(sequence_y))
        sample_weights.extend([sample_weight(gas, look_back = new_look_back, to_weight=to_weight)]*len(sequence_X))  # disable sample weiting
    return np.array(big_sequence_X), np.array(big_sequence_y), np.array(sample_weights)

In [ ]:
def train_test_RNN(df_: pd.DataFrame, look_back: int = 50, n_components: int = 5, do_PCA: bool = True, to_weight: bool = False, start: int = 7) -> (list, list, list, list):
    '''
    returns 6 lists for train/test/weights x/y. x and y are splitted by meas_cycle 7'''
    if not 'meas_cycle' in df_.columns:  # вставить защиту от дурака и слишком длинного look_back
        print("wrong dataframe: no meas_cycle column")
        return None
    else:
        #do_PCA_ = True if do_PCA == 'yes' else False
        #n_splits = len(df_.meas_cycle.unique())
        train_X, test_X, train_y, test_y = [], [], [], []
        train_y_ = df_.loc[df_['meas_cycle']<=start].iloc[:,402:405].values
        test_y_ = df_.loc[df_['meas_cycle']>start].iloc[:,402:405].values  # df_['meas_cycle']==split+1
        train_gas = df_.loc[df_['meas_cycle']<=start].loc[:, 'gas'].values
        test_gas = df_.loc[df_['meas_cycle']>start].loc[:, 'gas'].values  # df_['meas_cycle']==split+1

        # PCA
        train = df_.loc[df_['meas_cycle']<=start].iloc[:,:402].values
        test = df_.loc[df_['meas_cycle']>start].iloc[:,:402].values  # df_['meas_cycle']==split+1
        train_pca, test_pca = scale_n_PCA(train, test, n_components=n_components, scale=True, scaler=MinMaxScaler(), do_PCA=do_PCA)

        # careful splitting
        train_X_, train_y__, train_sw = create_RNN_sequences_for_multiple_gases(X = train_pca, y = train_y_, gas_type = train_gas, look_back = look_back, to_weight=to_weight)
        train_X.append(train_X_)
        train_y.append(train_y__)
        
        test_X_, test_y__, test_sw = create_RNN_sequences_for_multiple_gases(X = test_pca, y = test_y_, gas_type = test_gas, look_back = look_back, to_weight=to_weight)
        test_X.append(test_X_)
        test_y.append(test_y__)

        return train_X, test_X, train_y, test_y, train_sw, test_sw

In [ ]:
def train_test_RNN_(df_: pd.DataFrame, look_back: int = 50, n_components: int = 5, start: int = 5, do_PCA: bool = True, to_weight: bool = False) -> (list, list, list, list):
    '''
    returns 4 lists for train/test x/y. Each element = split
    start - measurement cycle to start with (each set of training data contains one more meas_cycle)'''
    if not 'meas_cycle' in df_.columns:  # вставить защиту от дурака и слишком длинного look_back
        print("wrong dataframe: no meas_cycle column")
        return None
    else:
        #do_PCA_ = True if do_PCA == 'yes' else False
        n_splits = len(df_.meas_cycle.unique())
        train_X, test_X, train_y, test_y = [], [], [], []
        for split in range(start,n_splits-1):  # starting from 1 cause the first meas cycle was cut
            train_y_ = df_.loc[df_['meas_cycle']<=split].iloc[:,402:405].values
            test_y_ = df_.loc[df_['meas_cycle']==split+1].iloc[:,402:405].values  # df_['meas_cycle']==split+1  df_['meas_cycle']>split
            train_gas = df_.loc[df_['meas_cycle']<=split].loc[:, 'gas'].values
            test_gas = df_.loc[df_['meas_cycle']==split+1].loc[:, 'gas'].values  # df_['meas_cycle']==split+1

            # PCA
            train = df_.loc[df_['meas_cycle']<=split].iloc[:,:402].values
            test = df_.loc[df_['meas_cycle']>split].iloc[:,:402].values  # df_['meas_cycle']==split+1
            train_pca, test_pca = scale_n_PCA(train, test, n_components=n_components, scale=True, scaler=MinMaxScaler(), do_PCA=do_PCA)

            # careful splitting
            train_X_, train_y__, train_sw = create_RNN_sequences_for_multiple_gases(X = train_pca, y = train_y_, gas_type = train_gas, look_back = look_back, to_weight=to_weight)
            train_X.append(train_X_)
            train_y.append(train_y__)
            
            test_X_, test_y__, test_sw = create_RNN_sequences_for_multiple_gases(X = test_pca, y = test_y_, gas_type = test_gas, look_back = look_back, to_weight=to_weight)
            test_X.append(test_X_)
            test_y.append(test_y__)

        return train_X, test_X, train_y, test_y, train_sw, test_sw

In [ ]:
def plot_history(history, params: dict) -> None:
    '''plots the history of NN training and saves it to a folder'''
    fig = px.line(pd.DataFrame(history).applymap(np.sqrt))
    #fig_title = str(f'{params}')
    annotations_keys = [
        dict(
            x=0.95,  # Adjust the x-coordinate for alignment
            y=0.88 - i * 0.06,  # Adjust the y-coordinate for vertical spacing
            xref="paper",
            yref="paper",
            text=str(key),
            showarrow=False,
            font_size=11,)
        for i, key in enumerate(params.keys())
    ]
    annotations_values = [dict(x=1,  # Adjust the x-coordinate for alignment
            y=0.88 - i * 0.06,  # Adjust the y-coordinate for vertical spacing
            xref="paper",
            yref="paper",
            text=str(value) if type(value) != float else str(round(value, 5)),
            showarrow=False,
            font_size=11)
        for i, value in enumerate(params.values())]
    fig.update_layout(xaxis_title = 'Epoch #',
                    yaxis_title = 'RMSE, ppm',
                    legend_title = '',
                    font_size = 20,
                    annotations = annotations_keys + annotations_values,
                    #title_text = fig_title,
                    #title_font_size = 9,
                    legend_orientation = 'h',
                    legend_x = 1,
                    legend_y = 1,
                    legend_xanchor = 'right',
                    legend_yanchor = 'top',
                    yaxis_nticks = 5,
                    margin = dict(t=30, b=0, r=0, l=10),
                    width = 1500,
                    height = 400)
    savepath = r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data\Feb 2021\NN training graphs".replace('\\', '/')
    savepath += "/" + "_".join([str(x) for x in params.values()]) + ".png"
    fig.write_image(savepath, format = 'png')

In [ ]:
def plt_CM(model, X, y, normalize, text: str = None):
    fig, ax = plt.subplots(figsize=(3,3), dpi=150)
    labels = [r"$acetone$", r'$H_{2}S$', r'$NO_{2}$', r'$air$']
    fig = ConfusionMatrixDisplay.from_estimator(estimator = model, X = X, y = y, normalize = normalize, xticks_rotation='horizontal', 
                                                ax = ax, cmap = 'viridis',
                                                display_labels=labels,
                                                )

    # Set title with F1 score and number of measurement cycles
    predictions = model.predict(X)
    f1_macro = f1_score(y, predictions, average='macro')
    title = f"Confusion Matrix \nF1 Score: {f1_macro:.2f}\n{text}"

    ax.set_title(title, fontsize=10)
    ax.set_xticklabels(labels, fontsize=8)
    ax.set_yticklabels(labels, fontsize=8)
    plt.show()

In [ ]:
train_X, test_X, train_y, test_y, train_SW, test_SW = train_test_RNN(DF, look_back = 50, n_components=20)

## Building the model

In [ ]:
import tensorflow as tf
def make_LSTM(X, y, n_LSTM_layers: int = 1, n_units: int = 64, dropout: float = 0, optimizer: tf.keras.optimizers = Adam, learning_rate: float = 0.001, loss: str = 'mean_squared_error'):
    input_shape = ((X[0]).shape[1], X[0].shape[2])
    output_shape = y[0].shape[1]
    to_return_sequences = lambda n_LSTM_layers_left: True if n_LSTM_layers_left > 1 else False

    model = Sequential()
    for n_LSTM_layers_left in range(n_LSTM_layers, 0, -1):
        model.add(LSTM(units=n_units, 
                       return_sequences=to_return_sequences(n_LSTM_layers_left), 
                       input_shape=input_shape, 
                       dropout = dropout, 
                       recurrent_dropout=dropout))
        n_units = n_units // 2 if n_units > 8 else n_units
    model.add(Dense(units=output_shape))

    model.compile(loss=loss, optimizer=optimizer(learning_rate=learning_rate), weighted_metrics = [])

    return model

def fit_LSTM(model, X_train, y_train, X_test, y_test, sw_train, sw_test, epochs: int = 100, batch_size: int = 32, return_history: bool = False):
    patience = 30
    callbacks = [EarlyStopping(monitor='val_loss', patience=patience, min_delta=0.1, restore_best_weights=True)]
    history = model.fit(
        X_train, y_train,
        validation_data = (X_test, y_test),
        #validation_data=(X_test, y_test, sw_test),
        #sample_weight = sw_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,
        callbacks = callbacks,
        shuffle = False,
        #workers = 32,
        #use_multiprocessing = True
        )
    #test_loss = model.evaluate(X_test, y_test, sample_weight = sw_test, batch_size = batch_size)
    y_pred = model.predict(X_test, batch_size = batch_size, verbose = 0,
                           #workers = 16, use_multiprocessing = True,
                           )
    test_rmse = mean_squared_error(y_test, y_pred, squared = False, multioutput = 'uniform_average')
    #test_rmse = mean_squared_error(y_test, y_pred, sample_weight= sw_test, squared = False, multioutput = 'uniform_average')
    ### taking history into account and trying to minimize the gap between train and validation loss
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    mse_epochs = [(x-y)**2 for x, y in zip(train_loss, val_loss)]
    history_slice = slice(-patience,len(mse_epochs),1)# slice(-3*patience//2,-patience//2,1)  # apparently it doesn't bug out if len(mse_epochs) < 2* patience
    historical_loss_rmse = np.sqrt(np.nanmean(mse_epochs[history_slice]))  # this slice was chosen as it contains the most optimized hyperparameters
    # here a new metric - historical_loss_rmse was introduced, that shows how far validation and training loss at the last patience epochs are

    hist_test_weights = [10,0]  # completely arbitrary parameter that sets up a weighted sum of traditional RMSE and HLRMSE with weights 2 and 1 respectively
    test_loss = (np.array([test_rmse, historical_loss_rmse]) * np.array(hist_test_weights)).sum() / sum(hist_test_weights)
    if return_history:
        return test_loss, history.history
    else:
        return test_loss, None


## tests

In [ ]:
test_X[0].shape

### NN regression

In [ ]:
params = dict(
    dedrifting_method = 'SavGol',
    envelope_choice = 'multienv',  # trial.suggest_categorical('dedrifting_method', ['multienv', 'topenv', 'None']),  #'multienv',  # 
    window_length = 250, #trial.suggest_int('window_length', 100, 700, step = 10),
    alpha = 0.022,
    look_back = 35, #trial.suggest_int('look_back', 5, 250),
    n_components = 110, # trial.suggest_int('n_components', 2, 70),
    do_PCA = True, # trial.suggest_categorical('do_PCA', [True, False]),
    n_LSTM_layers = 1, #trial.suggest_int('n_LSTM_layers', 1, 3),
    n_units = 35, # trial.suggest_int('n_units', 4, 256, log=True),
    dropout = 0.2, # trial.suggest_float('dropout', 0.05, 0.5, log=True),
    learning_rate= 0.02, #trial.suggest_float('learning_rate', 0.0001, 0.1, log=True),
    epochs = 200, #trial.suggest_int('epochs', 50, 300),
    batch_size = 128, #trial.suggest_categorical('batch_size', [8, 16, 32, 64, 128, 256]),
    )
DF = load_full_dedrifted_dataset(dedrifting_method=params['dedrifting_method'], envelope_choice=params['envelope_choice'], window_length=params['window_length'])
# Split data and perform PCA
train_X, test_X, train_y, test_y, train_SW, test_SW = train_test_RNN(DF, 
                                                    look_back= params['look_back'],
                                                    n_components= params['n_components'],
                                                    do_PCA= params['do_PCA'])
model = make_LSTM(train_X, 
                    train_y, 
                    optimizer = Adam, 
                    loss = 'mean_squared_error',
                    n_LSTM_layers = params['n_LSTM_layers'],
                    n_units = params['n_units'],
                    dropout = params['dropout'],
                    learning_rate = params['learning_rate'],)
# train the model
mse = []
#for X_train, X_test, y_train, y_test in zip(train_X[::-1], test_X[::-1], train_y[::-1], test_y[::-1]):  # reversed the order so the the trial be pruned earlier and more consistent
mse_, history_ = fit_LSTM(
    model, train_X[0], train_y[0], test_X[0], test_y[0], train_SW, test_SW,
    epochs = params['epochs'],
    batch_size= params['batch_size'], 
    return_history=True)
plot_history(history_, params)
#mse.append(mse_) 
print(mse_)


In [ ]:
y_pred = model.predict(test_X[0], batch_size = params['batch_size'], 
                           #workers = 16, use_multiprocessing = True,
                           )
test_rmse = mean_squared_error(test_y[0], y_pred, sample_weight= test_SW, squared = False, multioutput = 'raw_values')
result_df = pd.DataFrame(data = test_rmse, index = DF.columns[402:405], columns = ["RMSE"]).T
result_df

In [ ]:
import plotly.express as px
fig = px.line(pd.DataFrame(history_).applymap(np.sqrt))
annotations_keys = [
    dict(
        x=0.95,  # Adjust the x-coordinate for alignment
        y=0.88 - i * 0.06,  # Adjust the y-coordinate for vertical spacing
        xref="paper",
        yref="paper",
        text=str(key),
        showarrow=False,
        font_size=11,)
    for i, key in enumerate(params.keys())
]
annotations_values = [dict(x=1,  # Adjust the x-coordinate for alignment
         y=0.88 - i * 0.06,  # Adjust the y-coordinate for vertical spacing
         xref="paper",
         yref="paper",
         text=str(value) if type(value) != float else str(round(value, 5)),
         showarrow=False,
         font_size=11)
    for i, value in enumerate(params.values())]

fig.update_layout(xaxis_title = 'Epoch #',
                  yaxis_title = 'RMSE, ppm',
                  legend_title = '',
                  annotations = annotations_keys + annotations_values,
                  font_size = 20,
                  legend_orientation = 'h',
                  legend_x = 1,
                  legend_y = 1,
                  legend_xanchor = 'right',
                  legend_yanchor = 'top',
                  yaxis_nticks = 5,
                  margin = dict(t=30, b=0, r=0, l=10),
                  width = 1500,
                  height = 400)

#### Classification

In [ ]:
train_X, test_X, train_y, test_y = train_test_TS_class(add_class_column(DF), n_components = 153, start = 4)
new_NO2 = add_class_column(dedrift_n_cut(load_data("NO2 "), 'exp', 'multienv', 285, alpha= 0.02172, gas="NO2"))
ntrain_X, ntest_X, ntrain_y, ntest_y = train_test_TS_class(new_NO2, n_components = 153, start = 3)

In [ ]:
{'dedrifting_method': 'exp',
 'window_length': 285,
 'alpha': 0.02171655699670033,
 'n_components': 153,
 'iterations': 1300,
 'depth': 6,
 'learning_rate': 0.009372428573829332,
 'l2_leaf_reg': 13.155579412870834}

In [ ]:
model = CatBoostClassifier(
    iterations= 1300,
    depth=6,
    learning_rate=0.00937,
    l2_leaf_reg=13,
    loss_function="MultiClass",
    eval_metric="Accuracy",
)
f1_macro_scores = []
for i in range(len(train_X)):
    model.fit(
        train_X[i], train_y[i], 
        eval_set = (test_X[i], test_y[i]), 
        use_best_model=True,
        silent=True,
        plot=False)
    
    predictions = model.predict(test_X[i])
    f1_macro_scores.append(f1_score(test_y[i], predictions, average='macro'))

    fig, ax = plt.subplots(figsize=(3,3), dpi=150)
    labels = [r"$acetone$", r'$H_{2}S$', r'$NO_{2}$', r'$air$']
    fig = ConfusionMatrixDisplay.from_estimator(estimator = model, X = test_X[i], y = test_y[i], normalize = 'pred', xticks_rotation='horizontal', 
                                                ax = ax, cmap = 'viridis',
                                                display_labels=labels)
    
    # Set title with F1 score and number of measurement cycles
    title = f"Confusion Matrix \nF1 Score: {f1_score(test_y[i], predictions, average='macro'):.2f}\n meas_cycle {(5+i)}"
    
    ax.set_title(title, fontsize=10)
    ax.set_xticklabels(labels, fontsize=8)
    ax.set_yticklabels(labels, fontsize=8)
    plt.show()

In [ ]:
ntrain_X[0].shape

In [ ]:
train_X[-1].shape

In [ ]:
np.concatenate((ntrain_X[0], train_X[-1])).shape

In [ ]:
model = CatBoostClassifier(
    iterations= 500,
    depth=6,
    learning_rate=0.014,
    l2_leaf_reg=4,
    loss_function="MultiClass",
    eval_metric="Accuracy",
)
f1_macro_scores = []
model.fit(
    train_X[-1], train_y[-1], 
    eval_set = (test_X[-1], test_y[-1]), 
    use_best_model=True,
    silent=True,
    plot=False)

predictions = model.predict(np.concatenate((test_X[-1],ntrain_X[0])))
f1_macro_scores.append(f1_score(np.concatenate((test_y[-1],ntrain_y[0])), predictions, average='macro'))

fig, ax = plt.subplots(figsize=(3,3), dpi=150)
labels = [r"$acetone$", r'$H_{2}S$', r'$NO_{2}$', r'$air$']
fig = ConfusionMatrixDisplay.from_estimator(estimator = model, X = np.concatenate((test_X[-1],ntrain_X[0])), y = np.concatenate((test_y[-1],ntrain_y[0])), normalize = 'pred', xticks_rotation='horizontal', 
                                            ax = ax, cmap = 'viridis',
                                            display_labels=labels)

# Set title with F1 score and number of measurement cycles
title = f"Confusion Matrix \nF1 Score: {f1_score(np.concatenate((test_y[-1],ntrain_y[0])), predictions, average='macro'):.2f}\n meas_cycle {(5+i)}"

ax.set_title(title, fontsize=10)
ax.set_xticklabels(labels, fontsize=8)
ax.set_yticklabels(labels, fontsize=8)
plt.show()

## Defining Optuna functions

In [ ]:
def objective_LSTM(trial):
    # Load_data
    data_load_params = dict(
        dedrifting_method = trial.suggest_categorical('dedrifting_method', ['SavGol', 'exp', "none"]), # 'SavGol', 
        )
    if data_load_params['dedrifting_method'] == 'SavGol':
        data_load_params['window_length'] = trial.suggest_int('window_length', 150, 500, step = 10)
        data_load_params['envelope_choice'] = trial.suggest_categorical('envelope_choice', ['multienv', 'topenv'])
        data_load_params['alpha'] = 1
    elif data_load_params['dedrifting_method'] == 'exp':
        data_load_params['alpha'] = trial.suggest_float('alpha', 0.001, 0.1, log = True)
        data_load_params['window_length'] = 1
        data_load_params['envelope_choice'] = trial.suggest_categorical('envelope_choice', ['multienv', 'topenv'])
    elif data_load_params['dedrifting_method'] == 'none':
        data_load_params['envelope_choice'] = 'none'
        data_load_params['window_length'] = 1
        data_load_params['alpha'] = 1
    
    params = dict(
        look_back = trial.suggest_int('look_back', 20, 57, log=False),
        n_components = trial.suggest_int('n_components', 15, 150),
        do_PCA = True,  # trial.suggest_categorical('do_PCA', [True, False]),
        n_LSTM_layers = 1,  # trial.suggest_int('n_LSTM_layers', 1, 2),
        n_units = trial.suggest_int('n_units', 16, 128, log=False),  # trial.suggest_categorical('n_units', [16, 32, 64, 96, 128]),  
        dropout = trial.suggest_float('dropout', 0.005, 0.5, log=False),  # 0.162
        learning_rate= trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
        epochs = 150,  # trial.suggest_int('epochs', 50, 300),
        batch_size = trial.suggest_categorical('batch_size', [64, 128, 256]),
        )
    

    DF = load_full_dedrifted_dataset(**data_load_params)
    # Split data and perform PCA
    train_X, test_X, train_y, test_y, train_SW, test_SW = train_test_RNN(
        DF, 
        look_back= params['look_back'],
        n_components= params['n_components'],
        do_PCA= params['do_PCA'],
        start=8)
    model = make_LSTM(train_X, 
                      train_y, 
                      optimizer = Adam, 
                      loss = 'mean_squared_error',
                      n_LSTM_layers = params['n_LSTM_layers'],
                      n_units = params['n_units'],
                      dropout = params['dropout'],
                      learning_rate = params['learning_rate'],)
    # train the model
    #mse = []
    #pruning_flag = 0
    #for X_train, X_test, y_train, y_test in zip(train_X[::-1], test_X[::-1], train_y[::-1], test_y[::-1]):  # reversed the order so the the trial be pruned earlier and more consistent
    rmse_, history_ = fit_LSTM(
        model, train_X[0], train_y[0], test_X[0], test_y[0], 
        train_SW, test_SW,
        epochs = params['epochs'],
        batch_size= params['batch_size'], 
        return_history=True)
    plot_history(history_, params)
    
    return rmse_

In [ ]:
def objective_CBR(trial):
    # Load_data
    data_load_params = dict(dedrifting_method= trial.suggest_categorical('dedrifting_method', ['exp', 'none']),  # 'SavGol', 
                            envelope_choice = trial.suggest_categorical('envelope_choice', ['multienv', 'topenv']),
                            )
    if data_load_params['dedrifting_method'] == 'SavGol':
        data_load_params['window_length'] = trial.suggest_int('window_length', 150, 400, step = 10)
    elif data_load_params['dedrifting_method'] == 'exp':
        data_load_params['alpha'] = trial.suggest_float('alpha', 0.001, 0.1, log = True)
    
    DF = load_full_dedrifted_dataset(**data_load_params)
    # Split data and perform PCA
    train_X, test_X, train_y, test_y = train_test_TS(DF, n_components=trial.suggest_int('n_components', 5, 150), start = 6)
    # train the model
    cb_params = {
        'iterations': trial.suggest_int('iterations', 100, 1800, step = 50),
        'depth': trial.suggest_int('depth', 3, 7),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.5, log = True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3, 25)
    }
    catboost_model = CatBoostRegressor(**cb_params, loss_function='MultiRMSE', verbose=0)
    pruning_callback = CatBoostPruningCallback(trial, "MultiRMSE")
    mse = []
    #pruning_flag = 0
    for X_train, X_test, y_train, y_test in zip(train_X[::-1], test_X[::-1], train_y[::-1], test_y[::-1]):  # reversed the order so the the trial be pruned earlier and more consistent
        catboost_model.fit(X_train, y_train, 
                           eval_set=[(X_test, y_test)], 
                           early_stopping_rounds=200, # need to implement good pruning by starting from the most information train|test pair and not calculate the whole sequence if this pair is pruned
                           callbacks=[pruning_callback],
                           verbose=0)
        pruning_callback.check_pruned()
        y_pred = catboost_model.predict(X_test)  # add native penalization (sample_weights) according to TLV_TWA values for gases, NO2 - 0.2, H2S - 1, Acet - 
        mse_ = mean_squared_error(y_test, y_pred, multioutput='raw_values', squared = False) # substitute this with native CatBoost stuff
        TLV_weighted_mean = np.mean([x/y for x, y in zip(mse_,[0.2, 1, 250])])  # Assuming column order to be NO2 H2S Acetone
        mse.append(TLV_weighted_mean) 
    return np.mean(mse_)  # np.mean(mse)

In [ ]:
def objective_CBC(trial):
    # Load_data
    data_load_params = dict(dedrifting_method= trial.suggest_categorical('dedrifting_method', ['SavGol', 'exp', None]),
                            envelope_choice = trial.suggest_categorical('envelope_choice', ['multienv', 'topenv']),
                            )
    if data_load_params['dedrifting_method'] == 'SavGol':
        data_load_params['window_length'] = trial.suggest_int('window_length', 150, 400, step = 10)
        data_load_params['alpha'] = 1
    elif data_load_params['dedrifting_method'] == 'exp':
        data_load_params['alpha'] = trial.suggest_float('alpha', 0.001, 0.1, log = True)
    DF = load_full_dedrifted_dataset(**data_load_params)
    # Split data and perform PCA
    train_X, test_X, train_y, test_y = train_test_TS_class(add_class_column(DF), trial.suggest_int('n_components', 5, 200), start = 7)
    # train the model
    cb_params = {
        'iterations': trial.suggest_int('iterations', 100, 2000, step = 50),
        'depth': trial.suggest_int('depth', 3, 7),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.7, log = True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3, 15)
    }
    
    catboost_model = CatBoostClassifier(**cb_params, loss_function="MultiClass", eval_metric="Accuracy", use_best_model=True, silent=True)
    pruning_callback = CatBoostPruningCallback(trial, "MultiClass")
    mse = []
    #pruning_flag = 0
    f1_macro_scores = []
    for X_train, X_test, y_train, y_test in zip(train_X[::-1], test_X[::-1], train_y[::-1], test_y[::-1]):  # reversed the order so the the trial be pruned earlier and more consistent
        catboost_model.fit(X_train, y_train, 
                           eval_set=[(X_test, y_test)], 
                           early_stopping_rounds=200, # need to implement good pruning by starting from the most information train|test pair and not calculate the whole sequence if this pair is pruned
                           callbacks=[pruning_callback],
                           verbose=0)
        pruning_callback.check_pruned()
        '''if not pruning_flag:
            pruning_callback.check_pruned()
            pruning_flag += 1
            if trial.should_prune():
                break'''
        y_pred = catboost_model.predict(X_test)  # add native penalization (sample_weights) according to TLV_TWA values for gases, NO2 - 0.2, H2S - 1, Acet - 
        f1_macro_scores.append(f1_score(y_test, y_pred, average='macro'))
    
    return np.mean(f1_macro_scores)  # np.mean(mse)

In [ ]:
import warnings
import optuna
warnings.filterwarnings("ignore")
study = optuna.create_study(direction= 'minimize',  # minimize  for regression minimize  for classification 'maximize'
                            pruner= optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=0, interval_steps=1, n_min_trials=1),
                            #optuna.pruners.SuccessiveHalvingPruner(min_resource='auto', reduction_factor=3, min_early_stopping_rate=4, bootstrap_count=0) 
                            # optuna.pruners.MedianPruner(n_warmup_steps=10),
                            )
study.optimize(objective_LSTM, n_trials=10000, timeout=8*60*60, show_progress_bar = False, n_jobs=-1)  #n_jobs=-1

# Print the optimized hyperparameters and their values
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

In [ ]:
# checking how the model with the best hyperparameters performs
best_params = study.best_params
DF = load_full_dedrifted_dataset(best_params['dedrifting_method'], best_params['envelope_choice'], best_params['window_length'], best_params['alpha'])  # 
train_X, test_X, train_y, test_y = train_test(DF, n_components=best_params['n_components'], start = 5)
catboost_model = CatBoostRegressor(iterations = best_params['iterations'], 
                                   depth = best_params['depth'],
                                   learning_rate = best_params['learning_rate'],
                                   loss_function='MultiRMSE', verbose=0)
mse = []
for X_train, X_test, y_train, y_test in zip(train_X, test_X, train_y, test_y):
    catboost_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)
    y_pred = catboost_model.predict(X_test)
    mse.append(mean_squared_error(y_test, y_pred, multioutput='raw_values', squared = False)) 

In [ ]:
# NN
best_params = dict(
        envelope_choice = 'multienv',
        dedrifting_method = 'exp',
        window_length = 350,
        alpha = 0.022,
        look_back = 109,
        n_components = 98,
        do_PCA = True, 
        n_LSTM_layers = 1,
        n_units = 87, 
        dropout = 0.095538, 
        learning_rate= 0.018, 
        epochs = 120, 
        batch_size = 16,)
best_params = best_params | study.best_params
DF = load_full_dedrifted_dataset(dedrifting_method=best_params['dedrifting_method'], envelope_choice=best_params['envelope_choice'], window_length=best_params['window_length'], alpha= best_params['alpha'])
train_X, test_X, train_y, test_y, train_SW, test_SW = train_test_RNN(DF, look_back= best_params['look_back'], n_components= best_params['n_components'], do_PCA= best_params['do_PCA'])
model = make_LSTM(train_X, train_y, optimizer = Adam, loss = 'mean_squared_error', n_LSTM_layers = best_params['n_LSTM_layers'], n_units = best_params['n_units'], dropout = best_params['dropout'],
                  learning_rate = best_params['learning_rate'],)
rmse_, history_ = fit_LSTM(model, train_X[0], train_y[0], test_X[0], test_y[0], train_SW, test_SW,
    epochs = best_params['epochs'], batch_size= best_params['batch_size'], return_history=True)
plot_history(history_, best_params)
print(rmse_)
y_pred = model.predict(test_X[0], batch_size = best_params['batch_size'], workers = 16, use_multiprocessing = True)
test_rmse = mean_squared_error(test_y[0], y_pred, sample_weight= test_SW, squared = False, multioutput = 'raw_values')
print(test_rmse)

In [ ]:
study.best_params

In [ ]:
model.evaluate(test_X[0], test_y[0], batch_size = best_params['batch_size'])

In [ ]:
plot_history(history_, best_params | dict(a=1))

In [ ]:
print(best_params)
pd.DataFrame(mse, index = np.arange(6,10), columns = 'NO2 H2S Acet'.split())

In [ ]:
optuna.visualization.plot_param_importances(study).update_layout(height = 300, margin = dict(t=50, l=20, r=10, b=20)).show()
optuna.visualization.plot_param_importances(study, target=lambda t: t.duration.total_seconds(), target_name="duration").update_layout(height = 300, 
                                                                                                                                      margin = dict(t=50, l=20, r=10, b=20)).show()
optuna.visualization.plot_slice(study).show()           

In [ ]:
optuna.visualization.plot_rank(study, params = ['look_back', 'window_length'])  # , plot_contour params=["n_LSTM_layers", "n_units"]

In [ ]:
fig = optuna.visualization.plot_slice(study)
fig.update_layout(margin = dict(t=50, l=50, r=10, b=20))
format_ = 'svg'
slice_savepath = r"C:\Users\Bahrs\YandexDisk\SK_LNM\zzz____LNM\a_Article on SWCNT thermocycling\Experimental data".replace("\\","/")
slice_savepath += "/" + "look_back_optimization_5_324" + "." + format_
fig.write_image(slice_savepath, format = format_)

## Save study

In [ ]:
study.trials_dataframe().to_csv("Optuna.25.09.csv", sep = '\t', decimal = ',')


## Checking Feb data with new April 2022 data

In [ ]:
def load_full_dedrifted_dataset_new(dedrifting_method: str = 'exp', envelope_choice: str = 'multienv', window_length: int = 350, alpha: float = 0.022):
    '''dedrifting method - choose one of `SavGol`, `exp`, `None`
    envelope_choice - choose one of `topenv`, `multienv`
    window_length - be careful not to exceed the sample size'''
    DF = pd.DataFrame()
    for gas, actual_gas, mc in zip(['NO2 ', 'H2S', 'Acet'], ['NO2', 'H2S', 'Acet'],[4,8,8]):
        df = dedrift_n_cut(load_data(gas), method=dedrifting_method, how=envelope_choice, window_length=window_length, alpha=alpha, gas=actual_gas)
        DF = pd.concat([DF, df.loc[df["meas_cycle"]==mc]], ignore_index=True)
    return DF

#### Classification

In [ ]:
bparams = {  # classification with catboost
    'dedrifting_method': 'exp',
    'envelope_choice': "topenv",
    'window_length': 285,  # 300, #  
    'alpha': 0.02171655699670033,
    'n_components': 153,
    'iterations': 1300,
    'depth': 6,
    'learning_rate': 0.009372428573829332,
    'l2_leaf_reg': 13.155579412870834}

start = 7

DFn = add_class_column(load_full_dedrifted_dataset_new(bparams['dedrifting_method'], bparams["envelope_choice"], bparams['window_length'], bparams['alpha']))
DF = add_class_column(load_full_dedrifted_dataset(bparams['dedrifting_method'], bparams["envelope_choice"], bparams['window_length'], bparams['alpha']))
DFn = DFn.assign(meas_cycle = np.zeros(shape=DFn.shape[0]))
DFn.NO2.plot().update_layout(height = 150, margin = dict(t=0,l=0,r=0,b=0)).show()

do_PCA = True

train_y_ = DF.loc[DF['meas_cycle']<=start].iloc[:,402:405].values
test_y_ = DFn.iloc[:,402:405].values

train_gas = DF.loc[DF['meas_cycle']<=start].loc[:, 'gas'].values
test_gas = DFn.loc[:, 'gas'].values  

train_y_class = DF.loc[DF['meas_cycle']<=start].loc[:,"class_"].values
test_y_class = DFn.loc[:,"class_"].values

# PCA
train = DF.loc[DF['meas_cycle']<=start].iloc[:,:402].values
test = DFn.iloc[:,:402].values  
train_X_class, test_X_class = scale_n_PCA(train, test, n_components=bparams['n_components'], scale=True, scaler=MinMaxScaler(), do_PCA=do_PCA)

In [ ]:
model = CatBoostClassifier(
    iterations= bparams["iterations"],
    depth=bparams["depth"],
    learning_rate=bparams["learning_rate"],
    l2_leaf_reg=bparams["l2_leaf_reg"],
    loss_function="MultiClassOneVsAll",
    eval_metric="TotalF1",
)
model.fit(
    train_X_class, train_y_class, 
    eval_set = (test_X_class, test_y_class), 
    use_best_model=True,
    silent=True,
    plot=False)
plt_CM(model, test_X_class, test_y_class, None, "2 months validation")

In [ ]:
# traditional class
train_X, test_X, train_y, test_y = train_test_TS_class(DF, n_components = bparams["n_components"], start = 7)
model = CatBoostClassifier(
    iterations= bparams["iterations"],
    depth=bparams["depth"],
    learning_rate=bparams["learning_rate"],
    l2_leaf_reg=bparams["l2_leaf_reg"],
    loss_function="MultiClassOneVsAll",
    eval_metric="TotalF1",
)
model.fit(
    train_X[0], train_y[0], 
    eval_set = (test_X[0], test_y[0]), 
    use_best_model=True,
    silent=True,
    plot=False)

In [ ]:
plt_CM(model, test_X[0], test_y[0], None, r"$\text{Exponential  } \alpha = 0.022$")

In [ ]:
90/96

#### CBR

In [ ]:
bparams = {
 'dedrifting_method': 'SavGol',
 "envelope_choice": "multienv",
 'window_length': 250,
 "alpha": 0.022,
 'n_components': 19,
 'iterations': 1700,
 'depth': 8,
 'learning_rate': 0.014,
 'l2_leaf_reg': 3}

DFn = add_class_column(load_full_dedrifted_dataset_new(bparams['dedrifting_method'], bparams["envelope_choice"], bparams['window_length'], bparams['alpha']))
DF = add_class_column(load_full_dedrifted_dataset(bparams['dedrifting_method'], bparams["envelope_choice"], bparams['window_length'], bparams['alpha']))
DFn = DFn.assign(meas_cycle = np.zeros(shape=DFn.shape[0]))
DFn.NO2.plot().update_layout(height = 150, margin = dict(t=0,l=0,r=0,b=0)).show()

do_PCA = True

train_y_ = DF.loc[DF['meas_cycle']<=7].iloc[:,402:405].values
test_y_ = DFn.iloc[:,402:405].values

# PCA
train = DF.loc[DF['meas_cycle']<=7].iloc[:,:402].values
test = DFn.iloc[:,:402].values  
train_X_class, test_X_class = scale_n_PCA(train, test, n_components=bparams['n_components'], scale=True, scaler=MinMaxScaler(), do_PCA=do_PCA)

In [ ]:
catboost_model = CatBoostRegressor(
    iterations= bparams["iterations"],
    depth=bparams["depth"],
    learning_rate=bparams["learning_rate"],
    l2_leaf_reg=bparams["l2_leaf_reg"],
    loss_function='MultiRMSE', verbose=0)
catboost_model.fit(
    train_X_class, train_y_, 
    eval_set=[(test_X_class, test_y_)], 
    early_stopping_rounds=200, # need to implement good pruning by starting from the most information train|test pair and not calculate the whole sequence if this pair is pruned
    verbose=0)
    
y_pred = catboost_model.predict(test_X_class)  # add native penalization (sample_weights) according to TLV_TWA values for gases, NO2 - 0.2, H2S - 1, Acet - 
mse_ = mean_squared_error(test_y_, y_pred, multioutput='raw_values', squared = False) # substitute this with native CatBoost stuff
result_df = pd.DataFrame(data = mse_, index = DF.columns[402:405], columns = ["RMSE"]).T
result_df

In [ ]:
#traditional
train_y_ = DF.loc[DF['meas_cycle']<=7].iloc[:,402:405].values
test_y_ = DF.loc[DF['meas_cycle']>7].iloc[:,402:405].values

# PCA
train = DF.loc[DF['meas_cycle']<=7].iloc[:,:402].values
test = DF.loc[DF['meas_cycle']>7].iloc[:,:402].values  
train_X_class, test_X_class = scale_n_PCA(train, test, n_components=bparams['n_components'], scale=True, scaler=MinMaxScaler(), do_PCA=do_PCA)

catboost_model = CatBoostRegressor(
    iterations= bparams["iterations"],
    depth=bparams["depth"],
    learning_rate=bparams["learning_rate"],
    l2_leaf_reg=bparams["l2_leaf_reg"],
    loss_function='MultiRMSE', verbose=0)
catboost_model.fit(
    train_X_class, train_y_, 
    eval_set=[(test_X_class, test_y_)], 
    early_stopping_rounds=200, # need to implement good pruning by starting from the most information train|test pair and not calculate the whole sequence if this pair is pruned
    verbose=0)
    
y_pred = catboost_model.predict(test_X_class)  # add native penalization (sample_weights) according to TLV_TWA values for gases, NO2 - 0.2, H2S - 1, Acet - 
mse_ = mean_squared_error(test_y_, y_pred, multioutput='raw_values', squared = False) # substitute this with native CatBoost stuff
result_df = pd.DataFrame(data = mse_, index = DF.columns[402:405], columns = ["RMSE"]).T
result_df

#### LSTM

In [ ]:
bparams = {  # LSTM
    'dedrifting_method': 'SavGol',
    'envelope_choice': 'multienv',
    'window_length': 300,
    'look_back': 29,
    'n_components': 112,
    'n_units': 32,
    'dropout': 0.15,
    'learning_rate': 0.010415946044372366,
    'batch_size': 128,
    'alpha': 0.1  # for compatibility
    }
DFn = add_class_column(load_full_dedrifted_dataset_new(bparams['dedrifting_method'], bparams["envelope_choice"], bparams['window_length'], bparams['alpha']))
DF = add_class_column(load_full_dedrifted_dataset(bparams['dedrifting_method'], bparams["envelope_choice"], bparams['window_length'], bparams['alpha']))
DFn = DFn.assign(meas_cycle = np.zeros(shape=DFn.shape[0]))
DFn.NO2.plot().update_layout(height = 150, margin = dict(t=0,l=0,r=0,b=0)).show()

do_PCA = True

train_y_ = DF.loc[DF['meas_cycle']<=8].iloc[:,402:405].values
test_y_ = DFn.iloc[:,402:405].values

train_gas = DF.loc[DF['meas_cycle']<=8].loc[:, 'gas'].values
test_gas = DFn.loc[:, 'gas'].values  

train_y_class = DF.loc[DF['meas_cycle']<=8].loc[:,"class_"].values
test_y_class = DFn.loc[:,"class_"].values

# PCA
train = DF.loc[DF['meas_cycle']<=8].iloc[:,:402].values
test = DFn.iloc[:,:402].values  
train_X_class, test_X_class = scale_n_PCA(train, test, n_components=bparams['n_components'], scale=True, scaler=MinMaxScaler(), do_PCA=do_PCA)

In [ ]:
# careful splitting
to_weight = False
train_X_lstm, train_y_lstm, train_sw = create_RNN_sequences_for_multiple_gases(X = train_X_class, y = train_y_, gas_type = train_gas, look_back = bparams['look_back'], to_weight=to_weight)
test_X_lstm, test_y_lstm, test_sw = create_RNN_sequences_for_multiple_gases(X = test_X_class, y = test_y_, gas_type = test_gas, look_back = bparams['look_back'], to_weight=to_weight)

model = make_LSTM(
    [train_X_lstm], 
    [train_y_lstm], 
    optimizer = Adam, 
    loss = 'mean_squared_error',
    n_LSTM_layers = 1,
    n_units = bparams['n_units'],
    dropout = bparams['dropout'],
    learning_rate = bparams['learning_rate'],)
# train the model
mse_, history_ = fit_LSTM(
    model, train_X_lstm, train_y_lstm, test_X_lstm, test_y_lstm, train_sw, test_sw,
    epochs = 150,
    batch_size= bparams['batch_size'], 
    return_history=True)
plot_history(history_, bparams)
y_pred = model.predict(test_X_lstm, batch_size = bparams['batch_size'], 
                           #workers = 16, use_multiprocessing = True,
                           )
test_rmse = mean_squared_error(test_y_lstm, y_pred, sample_weight= test_sw, squared = False, multioutput = 'raw_values')
result_df = pd.DataFrame(data = test_rmse, index = DF.columns[402:405], columns = ["RMSE"]).T
result_df

In [ ]:
# traditional
train_X, test_X, train_y, test_y, train_sw, test_sw = train_test_RNN(DF, 
                                                    look_back= bparams['look_back'],
                                                    n_components= bparams['n_components'],
                                                    do_PCA= True,
                                                    start= 7)
model = make_LSTM(
    train_X, 
    train_y, 
    optimizer = Adam, 
    loss = 'mean_squared_error',
    n_LSTM_layers = 1,
    n_units = bparams['n_units'],
    dropout = bparams['dropout'],
    learning_rate = bparams['learning_rate'],)
# train the model
mse = []
#for X_train, X_test, y_train, y_test in zip(train_X[::-1], test_X[::-1], train_y[::-1], test_y[::-1]):  # reversed the order so the the trial be pruned earlier and more consistent
mse_, history_ = fit_LSTM(
    model, train_X[0], train_y[0], test_X[0], test_y[0], train_sw, test_sw,
    epochs = 200,
    batch_size= bparams['batch_size'], 
    return_history=True)
plot_history(history_, bparams)
y_pred = model.predict(test_X[0], batch_size = bparams['batch_size'], 
                           #workers = 16, use_multiprocessing = True,
                           )
test_rmse = mean_squared_error(test_y[0], y_pred, sample_weight= test_sw, squared = False, multioutput = 'raw_values')
result_df = pd.DataFrame(data = test_rmse, index = DF.columns[402:405], columns = ["RMSE"]).T
result_df

## Playground

### smoothing methods illustration

In [ ]:
data = load_data("NO2").I.iloc[201::402]
X = (data.index - data.index[0]).total_seconds().values/60
y = data.values*1000

In [ ]:
sg = SavGol(y, window_length = 250) + y.mean()
exp = y - forward_backward_exponential_smoothing(y, alpha=0.02) + y.mean()


In [ ]:
from re import match
gas = 'NO2'
def extract_gas_protocol(df: pd.DataFrame) -> (np.array, np.array):
    try:
        dfg = df.copy(deep=True)
        dfg = dfg.set_index((dfg.index - dfg.index[0]).total_seconds().values/60)
        dfg = dfg.assign(new_gas = dfg['MFC_target'].rolling(window=3).apply(lambda x: np.NaN if x[0] == x[1] == x[2] else x[1], raw = True))
        dfg = dfg.dropna()
        return [dfg.index[0]] + dfg.index.to_list() + [df.index[-1]], [df.MFC_target.iloc[0]] + dfg.new_gas.to_list() + [df.MFC_target.iloc[-1]]
    
    except AttributeError:
        print("****************\nTime and MFC_target are not in df.columns\n****************")
def color_with_opacity(color: str, opacity: float = 0.2) -> str:
    '''Takes a color of any format (hex, rgb) returns rgba with given opacity'''
    # Check if the input color is in hex format
    if color.startswith("#"):
        r, g, b = px.colors.hex_to_rgb(color)
    else:
        # Check if the input color is in 'rgb(r, g, b)' format
        rgb_match = match(r'rgb\((\d+), (\d+), (\d+)\)', color)
        if rgb_match:
            r, g, b = map(int, rgb_match.groups())
        else:
            raise ValueError("Invalid color format. Supported formats: hex or 'rgb(r, g, b)'.")

    rgba_color = f'rgba({r}, {g}, {b}, {opacity})'    
    return rgba_color
def create_gas_rectangles(gas_index: list, gas_values: list, colorscale, **kwargs) -> list:
    '''takes the index and values from extract_gas_protocol function and returns the list of go.layout.Shape dictionaries'''
    rectangles = []
    unique_gas_values = list(set(gas_values))
    colors = dict(zip(unique_gas_values, px.colors.sample_colorscale(colorscale, minmax_scale(unique_gas_values))))
    kwargs.setdefault('yref', 'paper')
    kwargs.setdefault('y0', 0)
    kwargs.setdefault('y1', 1)
    for i in range(0, len(gas_index) - 1, 2):
        text = '' if gas_values[i]==0 else f'<b>{gas_values[i]:.0f}</b>'  # gas is a global variable  #  ppm {target_gas_dict[gas]}
        color = 'rgba(255,255,250,0.3)' if gas_values[i]==0 else color_with_opacity(colors[gas_values[i]], 0.5)  # as well as target_gas_dict
        rectangle = dict(type = 'rect', 
                         xref= 'x', yref= kwargs['yref'], 
                         x0= gas_index[i], x1= gas_index[i+1], y0= kwargs['y0'], y1=kwargs["y1"], 
                         fillcolor = color, line_width= 0, 
                         label = dict(text= text, textangle= -0, xanchor= 'center', textposition= 'bottom center', padding= 20, font_color='black'),
                         layer = 'above',
                         )
        rectangles.append(go.layout.Shape(rectangle))
    return rectangles

In [ ]:
rectangles = create_gas_rectangles(*extract_gas_protocol(load_data("NO2")), 'Sunset_r')

In [ ]:
fig = go.Figure()
plotly = px.colors.qualitative.Plotly
fig.add_trace(go.Scatter(x = X, y = y, name= 'raw data 25 V envelope', line_color = plotly[1]))
fig.add_trace(go.Scatter(x = X, y = exp, name= 'exponential smoothing <br>baseline compensation', line_color = plotly[0]))
fig.add_trace(go.Scatter(x = X, y = sg, name= 'Savitzky-Golay smoothing <br>baseline compensation', line_color = plotly[5]))
fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)
fig.update_traces(line_width = 3)
fig.update_layout(template = 'simple_white',
                  font_family = "Times New Roman",
                  font_size = 22,
                  yaxis_nticks = 3,
                  xaxis_nticks = 5,
                  yaxis_title = "<b>Current</b>, mA",
                  xaxis_title = "<b>Time</b>, min",
                  legend_orientation = 'v',
                  legend_xanchor = 'right',
                  legend_x = 1,
                  legend_yanchor = 'bottom',
                  legend_y = 0.14,
                  width = 1122,
                  margin = dict(t=5, b=5, r=5, l=5),
                  shapes =rectangles,
                  yaxis_range = [16.2, 18.8],
                  xaxis_range = [0, 945])
fig.show()

In [ ]:
fig.write_image("../Graphs/Suppl/Smoothing illustration.png", scale = 2)

In [ ]:
type(DF.iloc[:,402:405].values)

In [ ]:
def create_RNN_sequences(data: np.ndarray, look_back: int = 50):
    num_samples, num_features = data.shape
    num_sequences = num_samples - look_back + 1
    
    sequences = np.zeros((num_sequences, look_back, num_features))
    
    for i in range(look_back):
        sequences[:, i] = data[i:num_samples - look_back + 1 + i]
    return sequences

In [ ]:
def train_test_RNN(df_: pd.DataFrame, look_back: int = 50, n_components: int = 5, start: int = 1) -> (list, list, list, list):
    '''start: int, meas_cycle to start with
    returns 4 lists for train/test x/y. Each element = split'''
    if not 'meas_cycle' in df_.columns:
        print("wrong dataframe: no meas_cycle column")
        return None
    else:
        n_splits = len(df_.meas_cycle.unique())
        train_X, test_X, train_y, test_y = [], [], [], []
        for split in range(start,n_splits-1):  # starting from 1 cause the first meas cycle was cut
            # PCA
            train = df_.loc[df_['meas_cycle']<=split].iloc[:,:402].values
            test = df_.loc[df_['meas_cycle']==split+1].iloc[:,:402].values
            train_pca, test_pca = scale_n_PCA(train, test, n_components=n_components, scale=True, scaler=MinMaxScaler(), do_PCA=True)
            train_X.append(train_pca)
            train_y.append(df_.loc[df_['meas_cycle']<=split].iloc[:,402:405].values)
            test_X.append(test_pca)
            test_y.append(df_.loc[df_['meas_cycle']==split+1].iloc[:,402:405].values)
        return train_X, test_X, train_y, test_y

In [ ]:
DF

In [ ]:
def quickplot(df, step: int = 6):
    stepping = slice(0, df.shape[0], step)
    y = df.loc[:,'I'].iloc[stepping]
    x = df.index[stepping]
    fig = go.Figure()
    fig.add_trace(go.Scattergl(x=x, y=y))
    fig.update_layout(hovermode = False)
    fig.show()

In [ ]:
quickplot(df)

In [ ]:
def extract_gas_protocol(df: pd.DataFrame) -> (np.array, np.array):
    try:
        dfg = df.assign(new_gas = df['MFC_target'].rolling(window=3).apply(lambda x: np.NaN if x[0] == x[1] == x[2] else x[1], raw = True))
        dfg.iloc[0, -1] = df.MFC_target.iloc[0]
        dfg.iloc[-1, -1] = df.MFC_target.iloc[-1]
        dfg = dfg.dropna()
        return dfg.index.to_list(), dfg.new_gas.to_list()
    
    except AttributeError:
        print("****************\nTime and MFC_target are not in df.columns\n****************")

In [ ]:
x, y = extract_gas_protocol(load_data('NO2 '))
fig = px.line(x=x, y=y)
fig.show()

In [ ]:
df.meas_cycle.loc[df["meas_cycle"]<5].plot()